<a href="https://colab.research.google.com/github/Janainareisnascimento/Predicitive-popcorn/blob/main/780_Predictivepopcorn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Predictive Popcorn - BSAN 780**


**Libraries and dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

file_path = '/content/movies_one_hot_encoding_202404231052.csv'
movies = pd.read_csv(file_path)
print(movies.head())
movies.info()


   consecutive_movie_id                         title  year  \
0                     1                    Toy Story   1995   
1                     2                      Jumanji   1995   
2                     3             Grumpier Old Men   1995   
3                     4            Waiting to Exhale   1995   
4                     5  Father of the Bride Part II   1995   

                                        genres  action  adventure  animation  \
0  Adventure|Animation|Children|Comedy|Fantasy       0          1          1   
1                   Adventure|Children|Fantasy       0          1          0   
2                               Comedy|Romance       0          0          0   
3                         Comedy|Drama|Romance       0          0          0   
4                                       Comedy       0          0          0   

   children  comedy  crime  ...  film-noir  horror  imax  musical  mystery  \
0         1       1      0  ...          0       0     0      

### **1) (EDA)**

---



In [ ]:

print(movies.describe())
print(movies.select_dtypes(include=['object']).apply(pd.Series.nunique))


       consecutive_movie_id       action    adventure    animation  \
count           9742.000000  9742.000000  9742.000000  9742.000000   
mean            4871.500000     0.187641     0.129645     0.062718   
std             2812.417495     0.390445     0.335930     0.242468   
min                1.000000     0.000000     0.000000     0.000000   
25%             2436.250000     0.000000     0.000000     0.000000   
50%             4871.500000     0.000000     0.000000     0.000000   
75%             7306.750000     0.000000     0.000000     0.000000   
max             9742.000000     1.000000     1.000000     1.000000   

          children       comedy        crime  documentary        drama  \
count  9742.000000  9742.000000  9742.000000  9742.000000  9742.000000   
mean      0.068158     0.385547     0.123075     0.045165     0.447649   
std       0.252031     0.486749     0.328541     0.207677     0.497277   
min       0.000000     0.000000     0.000000     0.000000     0.000000   

**Report 3**

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Function to run SQL queries
def run_query(query):
    conn = sqlite3.connect('/content/Movies.db')
    result = pd.read_sql_query(query, conn)
    conn.close()
    return result

# Function to execute a command that does not return a dataframe (like CREATE TABLE)
def run_command(command):
    conn = sqlite3.connect('Movies.db')
    cur = conn.cursor()
    cur.execute(command)
    conn.commit()
    conn.close()





**Code 1: Processing.sql**

In [ ]:
import sqlite3
import pandas as pd


conn = sqlite3.connect('/content/Movies.db')
cur = conn.cursor()


def execute_sql_command(command):
    cur.execute(command)
    conn.commit()


def fetch_sql_query(query):
    return pd.read_sql_query(query, conn)

# SQL commands to create tables and split genres
sql_commands = """
DROP TABLE IF EXISTS New_Movie;
CREATE TABLE New_Movie AS
SELECT movieID,
SUBSTR(title, 1 , INSTR(title,'(1')-1) AS title,
SUBSTR(title, -5, 4) AS year,
genres
FROM Movies;

DROP TABLE IF EXISTS get_first_genre;
CREATE TABLE get_first_genre AS
SELECT movieID, title, year, genres,
CASE
   WHEN genres like '%|%' THEN SUBSTR(genres, 1, INSTR(genres,'|')-1)
   ELSE genres
END AS genres1
FROM New_Movie;

DROP TABLE IF EXISTS get_second_genre;
CREATE TABLE get_second_genre AS
SELECT movieID, title, year, genres, genres1,
CASE
   WHEN genres like '%|%|%' THEN SUBSTR(REPLACE(genres, genres1 || '|', ''), 1, INSTR(REPLACE(genres, genres1 || '|', ''), '|')-1)
   WHEN genres like '%|%' THEN SUBSTR(genres, INSTR(genres, '|')+1, LENGTH(genres))
   ELSE NULL
END AS genres2
FROM get_first_genre;

DROP TABLE IF EXISTS get_third_genre;
CREATE TABLE get_third_genre AS
SELECT movieID, title, year, genres, genres1, genres2,
CASE
   WHEN genres like '%|%|%|%' THEN SUBSTR(REPLACE(genres, genres1 || '|' || genres2|| '|', ''), 1, INSTR(REPLACE(genres, genres1 || '|' || genres2|| '|', ''), '|')-1)
   WHEN genres like '%|%|%' THEN SUBSTR(REPLACE(genres, genres1 || '|' || genres2|| '|', ''), 1, LENGTH(genres))
   ELSE NULL
END AS genres3
FROM get_second_genre;

DROP TABLE IF EXISTS get_fourth_genre;
CREATE TABLE get_fourth_genre AS
SELECT movieID, title, year, genres, genres1, genres2, genres3,
CASE
   WHEN genres like '%|%|%|%|%' THEN SUBSTR(REPLACE(genres, genres1 || '|' || genres2|| '|' || genres3 || '|', ''), 1, INSTR(REPLACE(genres, genres1 || '|' || genres2|| '|' || genres3 || '|', ''), '|')-1)
   WHEN genres like '%|%|%|%' THEN SUBSTR(REPLACE(genres, genres1 || '|' || genres2|| '|' || genres3 || '|', ''), 1, LENGTH(genres))
   ELSE NULL
END AS genres4
FROM get_third_genre;

DROP TABLE IF EXISTS get_fifth_genre;
CREATE TABLE get_fifth_genre AS
SELECT movieID, title, year, genres, genres1, genres2, genres3, genres4,
CASE
   WHEN genres like '%|%|%|%|%|%' THEN SUBSTR(REPLACE(genres, genres1 || '|' || genres2|| '|' || genres3 || '|' || genres4 || '|', ''), 1, INSTR(REPLACE(genres, genres1 || '|' || genres2|| '|' || genres3 || '|' || genres4 || '|', ''), '|')-1)
   WHEN genres like '%|%|%|%|%' THEN SUBSTR(REPLACE(genres, genres1 || '|' || genres2|| '|' || genres3 || '|' || genres4 || '|', ''), 1, LENGTH(genres))
   ELSE NULL
END AS genres5
FROM get_fourth_genre;

DROP TABLE IF EXISTS get_sixth_genre;
CREATE TABLE get_sixth_genre AS
SELECT movieID, title, year, genres, genres1, genres2, genres3, genres4, genres5,
CASE
   WHEN genres like '%|%|%|%|%|%|%' THEN SUBSTR(REPLACE(genres, genres1 || '|' || genres2|| '|' || genres3 || '|' || genres4 || '|' || genres5 || '|', ''), 1, INSTR(REPLACE(genres, genres1 || '|' || genres2|| '|' || genres3 || '|' || genres4 || '|' || genres5 || '|', ''), '|')-1)
   WHEN genres like '%|%|%|%|%|%' THEN SUBSTR(REPLACE(genres, genres1 || '|' || genres2|| '|' || genres3 || '|' || genres4 || '|' || genres5 || '|', ''), 1, LENGTH(genres))
   ELSE NULL
END AS genres6
FROM get_fifth_genre;

DROP TABLE IF EXISTS get_seventh_genre;
CREATE TABLE get_seventh_genre AS
SELECT movieID, title, year, genres, genres1, genres2, genres3, genres4, genres5, genres6,
CASE
   WHEN genres like '%|%|%|%|%|%|%|%' THEN SUBSTR(REPLACE(genres, genres1 || '|' || genres2|| '|' || genres3 || '|' || genres4 || '|' || genres5 || '|' || genres6 || '|', ''), 1, INSTR(REPLACE(genres, genres1 || '|' || genres2|| '|' || genres3 || '|' || genres4 || '|' || genres5 || '|' || genres6 || '|', ''), '|')-1)
   WHEN genres like '%|%|%|%|%|%|%' THEN SUBSTR(REPLACE(genres, genres1 || '|' || genres2|| '|' || genres3 || '|' || genres4 || '|' || genres5 || '|' || genres6 || '|', ''), 1, LENGTH(genres))
   ELSE NULL
END AS genres7
FROM get_sixth_genre;

DROP TABLE IF EXISTS get_eighth_genre;
CREATE TABLE get_eighth_genre AS
SELECT movieID, title, year, genres, genres1, genres2, genres3, genres4, genres5, genres6, genres7,
CASE
   WHEN genres like '%|%|%|%|%|%|%|%|%' THEN SUBSTR(REPLACE(genres, genres1 || '|' || genres2|| '|' || genres3 || '|' || genres4 || '|' || genres5 || '|' || genres6 || '|' || genres7 || '|', ''), 1, INSTR(REPLACE(genres, genres1 || '|' || genres2|| '|' || genres3 || '|' || genres4 || '|' || genres5 || '|' || genres6 || '|' || genres7 || '|', ''), '|')-1)
   WHEN genres like '%|%|%|%|%|%|%|%' THEN SUBSTR(REPLACE(genres, genres1 || '|' || genres2|| '|' || genres3 || '|' || genres4 || '|' || genres5 || '|' || genres6 || '|' || genres7 || '|', ''), 1, LENGTH(genres))
   ELSE NULL
END AS genres8
FROM get_seventh_genre;

DROP TABLE IF EXISTS get_ninth_genre;
CREATE TABLE get_ninth_genre AS
SELECT movieID, title, year, genres, genres1, genres2, genres3, genres4, genres5, genres6, genres7, genres8,
CASE
   WHEN genres like '%|%|%|%|%|%|%|%|%|%' THEN SUBSTR(REPLACE(genres, genres1 || '|' || genres2|| '|' || genres3 || '|' || genres4 || '|' || genres5 || '|' || genres6 || '|' || genres7 || '|' || genres8 || '|', ''), 1, INSTR(REPLACE(genres, genres1 || '|' || genres2|| '|' || genres3 || '|' || genres4 || '|' || genres5 || '|' || genres6 || '|' || genres7 || '|' || genres8 || '|', ''), '|')-1)
   WHEN genres like '%|%|%|%|%|%|%|%|%' THEN SUBSTR(REPLACE(genres, genres1 || '|' || genres2|| '|' || genres3 || '|' || genres4 || '|' || genres5 || '|' || genres6 || '|' || genres7 || '|' || genres8 || '|', ''), 1, LENGTH(genres))
   ELSE NULL
END AS genres9
FROM get_eighth_genre;

DROP TABLE IF EXISTS get_tenth_genre;
CREATE TABLE get_tenth_genre AS
SELECT movieID, title, year, genres, genres1, genres2, genres3, genres4, genres5, genres6, genres7, genres8, genres9,
CASE
   WHEN genres like '%|%|%|%|%|%|%|%|%|%' THEN REPLACE(genres, genres1 || '|' || genres2|| '|' || genres3 || '|' || genres4 || '|' || genres5 || '|' || genres6 || '|' || genres7 || '|' || genres8 || '|' || genres9 || '|', '')
   ELSE NULL
END AS genres10
FROM get_ninth_genre;
"""


for command in sql_commands.strip().split(';'):
    if command.strip():
        execute_sql_command(command)


all_genres_df = fetch_sql_query("""
SELECT genres1 AS genres FROM get_first_genre
UNION
SELECT genres2 AS genres FROM get_second_genre
UNION
SELECT genres3 AS genres FROM get_third_genre
UNION
SELECT genres4 AS genres FROM get_fourth_genre
UNION
SELECT genres5 AS genres FROM get_fifth_genre
UNION
SELECT genres6 AS genres FROM get_sixth_genre
UNION
SELECT genres7 AS genres FROM get_seventh_genre
UNION
SELECT genres8 AS genres FROM get_eighth_genre
UNION
SELECT genres9 AS genres FROM get_ninth_genre
UNION
SELECT genres10 AS genres FROM get_tenth_genre
""")

print(all_genres_df)


conn.close()


                genres
0                 None
1   (no genres listed)
2               Action
3            Adventure
4            Animation
5             Children
6               Comedy
7                Crime
8          Documentary
9                Drama
10             Fantasy
11           Film-Noir
12              Horror
13                IMAX
14             Musical
15             Mystery
16             Romance
17              Sci-Fi
18            Thriller
19                 War
20             Western


**Code 2: BSAN 780 SQL part 3.28 .sql**

In [ ]:
import sqlite3
import pandas as pd


conn = sqlite3.connect('/content/Movies.db')
cur = conn.cursor()


def execute_sql_command(command):
    cur.execute(command)
    conn.commit()

# SQL commands to create tables and split genres, including one-hot encoding
sql_commands = """
DROP TABLE IF EXISTS New_Movie;
CREATE TABLE New_Movie AS
SELECT movieID,
SUBSTR(title, 1, LENGTH(title) - LENGTH(SUBSTR(title, INSTR(title, '(')))) AS title,
SUBSTR(title, LENGTH(title) - 4, 4) AS year,
genres
FROM Movies;

DROP TABLE IF EXISTS get_first_genre;
CREATE TABLE get_first_genre AS
SELECT movieID, title, year, genres,
CASE
   WHEN genres like '%|%' THEN SUBSTR(genres, 1, INSTR(genres,'|')-1)
   ELSE genres
END AS genres1
FROM New_Movie;

DROP TABLE IF EXISTS get_second_genre;
CREATE TABLE get_second_genre AS
SELECT movieID, title, year, genres, genres1,
CASE
   WHEN genres like '%|%|%' THEN SUBSTR(REPLACE(genres, genres1 || '|', ''), 1, INSTR(REPLACE(genres, genres1 || '|', ''), '|')-1)
   ELSE SUBSTR(genres, INSTR(genres, '|')+1)
END AS genres2
FROM get_first_genre;

-- Additional genres from third to tenth
DROP TABLE IF EXISTS get_third_genre;
CREATE TABLE get_third_genre AS
SELECT movieID, title, year, genres, genres1, genres2,
CASE
   WHEN genres like '%|%|%|%' THEN SUBSTR(REPLACE(genres, genres1 || '|' || genres2|| '|', ''), 1, INSTR(REPLACE(genres, genres1 || '|' || genres2|| '|', ''), '|')-1)
   ELSE NULL
END AS genres3
FROM get_second_genre;

-- Continue this pattern for genres 4 to 10

DROP TABLE IF EXISTS get_fourth_genre;
CREATE TABLE get_fourth_genre AS
SELECT movieID, title, year, genres, genres1, genres2, genres3,
CASE
   WHEN genres like '%|%|%|%|%' THEN SUBSTR(REPLACE(genres, genres1 || '|' || genres2 || '|' || genres3 || '|', ''), 1, INSTR(REPLACE(genres, genres1 || '|' || genres2 || '|' || genres3 || '|', ''), '|')-1)
   ELSE NULL
END AS genres4
FROM get_third_genre;

DROP TABLE IF EXISTS get_fifth_genre;
CREATE TABLE get_fifth_genre AS
SELECT movieID, title, year, genres, genres1, genres2, genres3, genres4,
CASE
   WHEN genres like '%|%|%|%|%|%' THEN SUBSTR(REPLACE(genres, genres1 || '|' || genres2 || '|' || genres3 || '|' || genres4 || '|', ''), 1, INSTR(REPLACE(genres, genres1 || '|' || genres2 || '|' || genres3 || '|' || genres4 || '|', ''), '|')-1)
   ELSE NULL
END AS genres5
FROM get_fourth_genre;

DROP TABLE IF EXISTS get_sixth_genre;
CREATE TABLE get_sixth_genre AS
SELECT movieID, title, year, genres, genres1, genres2, genres3, genres4, genres5,
CASE
   WHEN genres like '%|%|%|%|%|%|%' THEN SUBSTR(REPLACE(genres, genres1 || '|' || genres2 || '|' || genres3 || '|' || genres4 || '|' || genres5 || '|', ''), 1, INSTR(REPLACE(genres, genres1 || '|' || genres2 || '|' || genres3 || '|' || genres4 || '|' || genres5 || '|', ''), '|')-1)
   ELSE NULL
END AS genres6
FROM get_fifth_genre;

DROP TABLE IF EXISTS get_seventh_genre;
CREATE TABLE get_seventh_genre AS
SELECT movieID, title, year, genres, genres1, genres2, genres3, genres4, genres5, genres6,
CASE
   WHEN genres like '%|%|%|%|%|%|%|%' THEN SUBSTR(REPLACE(genres, genres1 || '|' || genres2 || '|' || genres3 || '|' || genres4 || '|' || genres5 || '|' || genres6 || '|', ''), 1, INSTR(REPLACE(genres, genres1 || '|' || genres2 || '|' || genres3 || '|' || genres4 || '|' || genres5 || '|' || genres6 || '|', ''), '|')-1)
   ELSE NULL
END AS genres7
FROM get_sixth_genre;

DROP TABLE IF EXISTS get_eighth_genre;
CREATE TABLE get_eighth_genre AS
SELECT movieID, title, year, genres, genres1, genres2, genres3, genres4, genres5, genres6, genres7,
CASE
   WHEN genres like '%|%|%|%|%|%|%|%|%' THEN SUBSTR(REPLACE(genres, genres1 || '|' || genres2 || '|' || genres3 || '|' || genres4 || '|' || genres5 || '|' || genres6 || '|' || genres7 || '|', ''), 1, INSTR(REPLACE(genres, genres1 || '|' || genres2 || '|' || genres3 || '|' || genres4 || '|' || genres5 || '|' || genres6 || '|' || genres7 || '|', ''), '|')-1)
   ELSE NULL
END AS genres8
FROM get_seventh_genre;

DROP TABLE IF EXISTS get_ninth_genre;
CREATE TABLE get_ninth_genre AS
SELECT movieID, title, year, genres, genres1, genres2, genres3, genres4, genres5, genres6, genres7, genres8,
CASE
   WHEN genres like '%|%|%|%|%|%|%|%|%|%' THEN SUBSTR(REPLACE(genres, genres1 || '|' || genres2 || '|' || genres3 || '|' || genres4 || '|' || genres5 || '|' || genres6 || '|' || genres7 || '|' || genres8 || '|', ''), 1, INSTR(REPLACE(genres, genres1 || '|' || genres2 || '|' || genres3 || '|' || genres4 || '|' || genres5 || '|' || genres6 || '|' || genres7 || '|' || genres8 || '|', ''), '|')-1)
   ELSE NULL
END AS genres9
FROM get_eighth_genre;

DROP TABLE IF EXISTS get_tenth_genre;
CREATE TABLE get_tenth_genre AS
SELECT movieID, title, year, genres, genres1, genres2, genres3, genres4, genres5, genres6, genres7, genres8, genres9,
CASE
   WHEN genres like '%|%|%|%|%|%|%|%|%|%|%' THEN SUBSTR(REPLACE(genres, genres1 || '|' || genres2 || '|' || genres3 || '|' || genres4 || '|' || genres5 || '|' || genres6 || '|' || genres7 || '|' || genres8 || '|' || genres9 || '|', ''), 1, INSTR(REPLACE(genres, genres1 || '|' || genres2 || '|' || genres3 || '|' || genres4 || '|' || genres5 || '|' || genres6 || '|' || genres7 || '|' || genres8 || '|' || genres9 || '|', ''), '|')-1)
   ELSE NULL
END AS genres10
FROM get_ninth_genre;

DROP TABLE IF EXISTS all_genres;
CREATE TABLE all_genres AS
SELECT genres1 AS genres FROM get_first_genre
UNION
SELECT genres2 FROM get_second_genre
UNION
SELECT genres3 FROM get_third_genre
UNION
SELECT genres4 FROM get_fourth_genre
UNION
SELECT genres5 FROM get_fifth_genre
UNION
SELECT genres6 FROM get_sixth_genre
UNION
SELECT genres7 FROM get_seventh_genre
UNION
SELECT genres8 FROM get_eighth_genre
UNION
SELECT genres9 FROM get_ninth_genre
UNION
SELECT genres10 FROM get_tenth_genre;

DROP TABLE IF EXISTS movies_one_hot_encoding;
CREATE TABLE movies_one_hot_encoding AS
SELECT
movieID,
title,
year,
genres,
CASE WHEN lower(genres) LIKE '%action%' THEN 1 ELSE 0 END AS action,
CASE WHEN lower(genres) LIKE '%adventure%' THEN 1 ELSE 0 END AS adventure,
CASE WHEN lower(genres) LIKE '%animation%' THEN 1 ELSE 0 END AS animation,
CASE WHEN lower(genres) LIKE '%children%' THEN 1 ELSE 0 END AS children,
CASE WHEN lower(genres) LIKE '%comedy%' THEN 1 ELSE 0 END AS comedy,
CASE WHEN lower(genres) LIKE '%crime%' THEN 1 ELSE 0 END AS crime,
CASE WHEN lower(genres) LIKE '%documentary%' THEN 1 ELSE 0 END AS documentary,
CASE WHEN lower(genres) LIKE '%drama%' THEN 1 ELSE 0 END AS drama,
CASE WHEN lower(genres) LIKE '%fantasy%' THEN 1 ELSE 0 END AS fantasy,
CASE WHEN lower(genres) LIKE '%film-noir%' THEN 1 ELSE 0 END AS 'film-noir',
CASE WHEN lower(genres) LIKE '%horror%' THEN 1 ELSE 0 END AS horror,
CASE WHEN lower(genres) LIKE '%imax%' THEN 1 ELSE 0 END AS imax,
CASE WHEN lower(genres) LIKE '%musical%' THEN 1 ELSE 0 END AS musical,
CASE WHEN lower(genres) LIKE '%mystery%' THEN 1 ELSE 0 END AS mystery,
CASE WHEN lower(genres) LIKE '%romance%' THEN 1 ELSE 0 END AS romance,
CASE WHEN lower(genres) LIKE '%sci-fi%' THEN 1 ELSE 0 END AS 'sci-fi',
CASE WHEN lower(genres) LIKE '%thriller%' THEN 1 ELSE 0 END AS thriller,
CASE WHEN lower(genres) LIKE '%war%' THEN 1 ELSE 0 END AS war,
CASE WHEN lower(genres) LIKE '%western%' THEN 1 ELSE 0 END AS western
FROM New_Movie;
"""


for command in sql_commands.split(';'):
    if command.strip():
        execute_sql_command(command)


movies_one_hot_encoding_df = fetch_sql_query("SELECT * FROM movies_one_hot_encoding;")
print(movies_one_hot_encoding_df.head())

ratings_df = fetch_sql_query("SELECT * FROM Ratings;")
print(ratings_df.head())


conn.close()


   movieID                         title  year  \
0        1                    Toy Story   1995   
1        2                      Jumanji   1995   
2        3             Grumpier Old Men   1995   
3        4            Waiting to Exhale   1995   
4        5  Father of the Bride Part II   1995   

                                        genres  action  adventure  animation  \
0  Adventure|Animation|Children|Comedy|Fantasy       0          1          1   
1                   Adventure|Children|Fantasy       0          1          0   
2                               Comedy|Romance       0          0          0   
3                         Comedy|Drama|Romance       0          0          0   
4                                       Comedy       0          0          0   

   children  comedy  crime  ...  film-noir  horror  imax  musical  mystery  \
0         1       1      0  ...          0       0     0        0        0   
1         1       0      0  ...          0       0     0      

**Code 3: gener_similarity.py**

In [68]:
import numpy as np
import pandas as pd
import sqlite3

conn = sqlite3.connect('/content/Movies.db')
cur = conn.cursor()

cur.execute('SELECT action, adventure, animation, children, comedy, crime, documentary, drama, fantasy, "film-noir", horror, imax, musical, mystery, romance, "sci-fi", thriller, war, western FROM movies_one_hot_encoding')
one_zero_vectors = np.array(cur.fetchall(), dtype=int)
print(one_zero_vectors.shape)

# Computing the similarity matrix using matrix multiplication
similarity_matrix = np.matmul(one_zero_vectors, np.transpose(one_zero_vectors))

# Filling diagonal with zeros to exclude self-similarity
np.fill_diagonal(similarity_matrix, 0)

# Creating a DataFrame from the similarity matrix
df = pd.DataFrame(similarity_matrix, columns=range(1, len(one_zero_vectors) + 1))
print(df)

# Finding the index of maximum similarity for each movie
df_max_match = df.idxmax(axis=1)

# Saving the results to CSV file
df_max_match.to_csv('results.csv', header=False)
conn.close()








(9742, 19)
      1     2     3     4     5     6     7     8     9     10    ...  9733  \
0        0     3     1     1     1     0     1     2     0     1  ...     2   
1        3     0     0     0     0     0     0     2     0     1  ...     0   
2        1     0     0     2     1     0     2     0     0     0  ...     1   
3        1     0     2     0     1     0     2     0     0     0  ...     1   
4        1     0     1     1     0     0     1     0     0     0  ...     1   
...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
9737     3     1     1     1     1     1     1     0     1     1  ...     3   
9738     3     1     1     1     1     0     1     0     0     0  ...     2   
9739     0     0     0     1     0     0     0     0     0     0  ...     0   
9740     1     0     0     0     0     1     0     0     1     1  ...     2   
9741     1     0     1     1     1     0     1     0     0     0  ...     1   

      9734  9735  9736  9737  9738  9739

**Cosine similarity**


In [ ]:
import numpy as np
import pandas as pd
import sqlite3
from sklearn.metrics.pairwise import cosine_similarity

conn = sqlite3.connect('/content/Movies.db')
cur = conn.cursor()

# Selecting the one-hot encoded genres from the database
cur.execute('SELECT action, adventure, animation, children, comedy, crime, documentary, drama, fantasy, "film-noir", horror, imax, musical, mystery, romance, "sci-fi", thriller, war, western FROM movies_one_hot_encoding')
one_zero_vectors = np.array(cur.fetchall(), dtype=int)

# Calculating cosine similarity
cos_sim_matrix = cosine_similarity(one_zero_vectors)

# Creating a DataFrame from the cosine similarity matrix
df_cos_sim = pd.DataFrame(cos_sim_matrix, columns=range(1, len(one_zero_vectors)+1), index=range(1, len(one_zero_vectors)+1))
print(df_cos_sim.head())

df_cos_sim.to_csv('cosine_similarity_results.csv')
conn.close()


       1         2         3         4         5     6         7         8     \
1  1.000000  0.774597  0.316228  0.258199  0.447214   0.0  0.316228  0.632456   
2  0.774597  1.000000  0.000000  0.000000  0.000000   0.0  0.000000  0.816497   
3  0.316228  0.000000  1.000000  0.816497  0.707107   0.0  1.000000  0.000000   
4  0.258199  0.000000  0.816497  1.000000  0.577350   0.0  0.816497  0.000000   
5  0.447214  0.000000  0.707107  0.577350  1.000000   0.0  0.707107  0.000000   

   9         10    ...      9733      9734      9735      9736  9737  \
1   0.0  0.258199  ...  0.447214  0.316228  0.316228  0.447214   0.0   
2   0.0  0.333333  ...  0.000000  0.000000  0.000000  0.000000   0.0   
3   0.0  0.000000  ...  0.353553  0.000000  0.500000  0.000000   0.0   
4   0.0  0.000000  ...  0.288675  0.408248  0.816497  0.000000   0.0   
5   0.0  0.000000  ...  0.500000  0.000000  0.707107  0.000000   0.0   

       9738      9739     9740      9741      9742  
1  0.670820  0.774597  0.00

**Code 4: dataset with new movie .sql**

In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('/content/Movies.db')
cur = conn.cursor()

def execute_sql_command(command):
    cur.execute(command)
    conn.commit()

sql_commands = """
DROP TABLE IF EXISTS New_Movie;
CREATE TABLE New_Movie AS
SELECT movieID,
       title,
       SUBSTR(title, LENGTH(title) - 4, 4) AS year,
       genres
FROM Movies;

-- Create a one-hot encoding for genres
DROP TABLE IF EXISTS movies_one_hot_encoding;
CREATE TABLE movies_one_hot_encoding AS
SELECT
    movieID as consecutive_movie_id,
    title,
    year,
    genres,
    CASE WHEN lower(genres) LIKE '%action%' THEN 1 ELSE 0 END AS action,
    CASE WHEN lower(genres) LIKE '%adventure%' THEN 1 ELSE 0 END AS adventure,
    -- Repeat for other genres
    CASE WHEN lower(genres) LIKE '%sci-fi%' THEN 1 ELSE 0 END AS 'sci-fi',
    CASE WHEN lower(genres) LIKE '%thriller%' THEN 1 ELSE 0 END AS thriller,
    CASE WHEN lower(genres) LIKE '%war%' THEN 1 ELSE 0 END AS war,
    CASE WHEN lower(genres) LIKE '%western%' THEN 1 ELSE 0 END AS western
FROM New_Movie;

-- Prepare crossjoin for similarity ratings
DROP TABLE IF EXISTS Userid;
CREATE TABLE Userid AS
SELECT DISTINCT userid FROM Ratings;

DROP TABLE IF EXISTS movieid;
CREATE TABLE movieid AS
SELECT DISTINCT movieID as consecutive_movie_id FROM New_Movie;

DROP TABLE IF EXISTS crossjoin;
CREATE TABLE crossjoin AS
SELECT * FROM Userid CROSS JOIN movieid;

DROP TABLE IF EXISTS similarity_ratings;
CREATE TABLE similarity_ratings AS
SELECT cj.userID, cj.consecutive_movie_id, r.rating
FROM crossjoin cj
LEFT JOIN Ratings r ON cj.userID = r.userID AND cj.consecutive_movie_id = r.movieID;

UPDATE similarity_ratings SET rating = 0 WHERE rating IS NULL;
"""

for command in sql_commands.split(';'):
    if command.strip():
        execute_sql_command(command)


movies_one_hot_encoding_df = pd.read_sql_query("SELECT * FROM movies_one_hot_encoding;", conn)
print(movies_one_hot_encoding_df.head())

similarity_ratings_df = pd.read_sql_query("SELECT * FROM similarity_ratings;", conn)
print(similarity_ratings_df.head())

conn.close()


   consecutive_movie_id                               title  year  \
0                     1                    Toy Story (1995)  1995   
1                     2                      Jumanji (1995)  1995   
2                     3             Grumpier Old Men (1995)  1995   
3                     4            Waiting to Exhale (1995)  1995   
4                     5  Father of the Bride Part II (1995)  1995   

                                        genres  action  adventure  sci-fi  \
0  Adventure|Animation|Children|Comedy|Fantasy       0          1       0   
1                   Adventure|Children|Fantasy       0          1       0   
2                               Comedy|Romance       0          0       0   
3                         Comedy|Drama|Romance       0          0       0   
4                                       Comedy       0          0       0   

   thriller  war  western  
0         0    0        0  
1         0    0        0  
2         0    0        0  
3         

In [ ]:
import sqlite3
import pandas as pd
import numpy as np


conn = sqlite3.connect('/content/Movies.db')
cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS New_Movie;')
cur.execute('''
CREATE TABLE New_Movie AS
SELECT movieID,
       SUBSTR(title, 1, LENGTH(title) - LENGTH(SUBSTR(title, INSTR(title, '(')))) AS title,
       SUBSTR(title, LENGTH(title) - 4, 4) AS year,
       genres
FROM Movies;
''')

cur.execute('DROP TABLE IF EXISTS movies_one_hot_encoding;')
cur.execute('''
CREATE TABLE movies_one_hot_encoding AS
SELECT
    ROW_NUMBER() OVER (ORDER BY movieID) AS consecutive_movie_id,
    title,
    year,
    genres,
    CASE WHEN lower(genres) LIKE '%action%' THEN 1 ELSE 0 END AS action,
    CASE WHEN lower(genres) LIKE '%adventure%' THEN 1 ELSE 0 END AS adventure,
    CASE WHEN lower(genres) LIKE '%animation%' THEN 1 ELSE 0 END AS animation,
    CASE WHEN lower(genres) LIKE '%children%' THEN 1 ELSE 0 END AS children,
    CASE WHEN lower(genres) LIKE '%comedy%' THEN 1 ELSE 0 END AS comedy,
    CASE WHEN lower(genres) LIKE '%crime%' THEN 1 ELSE 0 END AS crime,
    CASE WHEN lower(genres) LIKE '%documentary%' THEN 1 ELSE 0 END AS documentary,
    CASE WHEN lower(genres) LIKE '%drama%' THEN 1 ELSE 0 END AS drama,
    CASE WHEN lower(genres) LIKE '%fantasy%' THEN 1 ELSE 0 END AS fantasy,
    CASE WHEN lower(genres) LIKE '%film-noir%' THEN 1 ELSE 0 END AS 'film-noir',
    CASE WHEN lower(genres) LIKE '%horror%' THEN 1 ELSE 0 END AS horror,
    CASE WHEN lower(genres) LIKE '%imax%' THEN 1 ELSE 0 END AS imax,
    CASE WHEN lower(genres) LIKE '%musical%' THEN 1 ELSE 0 END AS musical,
    CASE WHEN lower(genres) LIKE '%mystery%' THEN 1 ELSE 0 END AS mystery,
    CASE WHEN lower(genres) LIKE '%romance%' THEN 1 ELSE 0 END AS romance,
    CASE WHEN lower(genres) LIKE '%sci-fi%' THEN 1 ELSE 0 END AS 'sci-fi',
    CASE WHEN lower(genres) LIKE '%thriller%' THEN 1 ELSE 0 END AS thriller,
    CASE WHEN lower(genres) LIKE '%war%' THEN 1 ELSE 0 END AS war,
    CASE WHEN lower(genres) LIKE '%western%' THEN 1 ELSE 0 END AS western
FROM New_Movie;
''')

conn.commit()  # Commit changes to the database

# Fetching the one-hot encoded genre data from the database
cur.execute('SELECT action, adventure, animation, children, comedy, crime, documentary, drama, fantasy, "film-noir", horror, imax, musical, mystery, romance, "sci-fi", thriller, war, western FROM movies_one_hot_encoding')
one_zero_vectors = np.array(cur.fetchall(), dtype=int)

# Displaying the shape of the fetched data
print(f"Shape of one-hot vectors: {one_zero_vectors.shape}")

# Computing the similarity matrix by calculating the dot product and then the transpose of the matrix
similarity_matrix = np.matmul(one_zero_vectors, np.transpose(one_zero_vectors))

# Zero out the diagonal to remove self-similarity
np.fill_diagonal(similarity_matrix, 0)

# Converting the similarity matrix to a DataFrame for better handling and viewing
df = pd.DataFrame(similarity_matrix, columns=range(1, len(one_zero_vectors)+1))
print(df)

# Finding the maximum match for each item
df_max_match = df.idxmax(axis=1)

df_max_match.to_csv('/content/results.csv', header=False)

conn.close()


Shape of one-hot vectors: (9742, 19)
      1     2     3     4     5     6     7     8     9     10    ...  9733  \
0        0     3     1     1     1     0     1     2     0     1  ...     2   
1        3     0     0     0     0     0     0     2     0     1  ...     0   
2        1     0     0     2     1     0     2     0     0     0  ...     1   
3        1     0     2     0     1     0     2     0     0     0  ...     1   
4        1     0     1     1     0     0     1     0     0     0  ...     1   
...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
9737     3     1     1     1     1     1     1     0     1     1  ...     3   
9738     3     1     1     1     1     0     1     0     0     0  ...     2   
9739     0     0     0     1     0     0     0     0     0     0  ...     0   
9740     1     0     0     0     0     1     0     0     1     1  ...     2   
9741     1     0     1     1     1     0     1     0     0     0  ...     1   

      9734  97

**Code 5: Rating_similarity_movie.py and Table for similarity.sql**

In [ ]:
import sqlite3
import pandas as pd


conn = sqlite3.connect('/content/Movies.db')
cur = conn.cursor()

def execute_sql_commands(commands):
    for command in commands:
        cur.execute(command)
        conn.commit()

commands = [
    "DROP TABLE IF EXISTS New_Movie;",
    """CREATE TABLE New_Movie AS
    SELECT
        movieID,
        ROW_NUMBER() OVER (ORDER BY movieID) AS consecutive_movie_id,
        SUBSTR(title, 1, LENGTH(title) - LENGTH(SUBSTR(title, INSTR(title, '(')))) AS title,
        SUBSTR(title, LENGTH(title) - 4, 4) AS year,
        genres
    FROM Movies;"""
]

execute_sql_commands(commands)

new_movie_df = pd.read_sql_query("SELECT * FROM New_Movie LIMIT 10;", conn)
print(new_movie_df)

# Ensuring the Ratings and New_Movie tables are properly created
commands = [
    "DROP TABLE IF EXISTS new_rating;",
    """CREATE TABLE new_rating AS
    SELECT r.userID, r.movieID, r.rating, nm.consecutive_movie_id
    FROM Ratings r
    JOIN New_Movie nm ON r.movieID = nm.movieID;"""
]

execute_sql_commands(commands)

new_rating_df = pd.read_sql_query("SELECT * FROM new_rating LIMIT 10;", conn)
print(new_rating_df)

conn.close()


   movieID  consecutive_movie_id                         title  year  \
0        1                     1                    Toy Story   1995   
1        2                     2                      Jumanji   1995   
2        3                     3             Grumpier Old Men   1995   
3        4                     4            Waiting to Exhale   1995   
4        5                     5  Father of the Bride Part II   1995   
5        6                     6                         Heat   1995   
6        7                     7                      Sabrina   1995   
7        8                     8                 Tom and Huck   1995   
8        9                     9                 Sudden Death   1995   
9       10                    10                    GoldenEye   1995   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3          

In [ ]:
import numpy as np
import pandas as pd
import sqlite3
from sklearn.metrics.pairwise import cosine_similarity


conn = sqlite3.connect('/content/Movies.db')
cur = conn.cursor()


cur.execute("SELECT rating FROM similarity_ratings WHERE rating IS NOT NULL;")
ratings_vector = np.array(cur.fetchall(), dtype=float)

# Checking if the shape of the ratings_vector fits the expected format
try:
    ratings_matrix = ratings_vector.reshape(610, 9742)  # Ensure these dimensions match your dataset
except ValueError:
    print("The shape of the ratings_vector does not fit 610 users by 9742 movies. Please adjust.")

# Replacing None values with 0
ratings_matrix = np.nan_to_num(ratings_matrix)
print(ratings_matrix.shape)
print(ratings_matrix[1, :])

# Calculating cosine similarity matrix
cos_sim = cosine_similarity(ratings_matrix)
np.fill_diagonal(cos_sim, 0)  # Set diagonal to 0 to exclude self-similarity
print(cos_sim.shape)

# Creating a DataFrame for better visualization
df = pd.DataFrame(cos_sim, columns=range(1, len(ratings_matrix) + 1))
print(df.head())

# Finding the most similar user for each user
most_similar_customer_list = df.idxmax(axis=1).tolist()

# Recommending movies based on similar users' preferences
recommended_movies = []
for user_id in range(1, len(ratings_matrix) + 1):
    cur.execute("SELECT consecutive_movie_id FROM similarity_ratings WHERE userID=? AND rating>=1", (user_id,))
    user_movies = cur.fetchall()
    user_movie_ids = [m[0] for m in user_movies]

    most_similar_user = most_similar_customer_list[user_id - 1]
    cur.execute("SELECT consecutive_movie_id FROM similarity_ratings WHERE userID=? AND rating>=4", (most_similar_user,))
    similar_user_movies = cur.fetchall()
    similar_user_movie_ids = [m[0] for m in similar_user_movies]

    # Recommending movies from the similar user that the current user hasn't rated
    recommendations = np.setdiff1d(similar_user_movie_ids, user_movie_ids).tolist()
    recommended_movies.append(recommendations)

print(recommended_movies[0])  # Print recommendations for the first user

conn.close()


(610, 9742)
[0. 0. 0. ... 0. 0. 0.]
(610, 610)
        1         2         3         4         5         6         7    \
0  0.000000  0.027283  0.059720  0.194395  0.129080  0.128152  0.158744   
1  0.027283  0.000000  0.000000  0.003726  0.016614  0.025333  0.027585   
2  0.059720  0.000000  0.000000  0.002251  0.005020  0.003936  0.000000   
3  0.194395  0.003726  0.002251  0.000000  0.128659  0.088491  0.115120   
4  0.129080  0.016614  0.005020  0.128659  0.000000  0.300349  0.108342   

        8         9         10   ...       601       602       603       604  \
0  0.136968  0.064263  0.016875  ...  0.080554  0.164455  0.221486  0.070669   
1  0.027257  0.000000  0.067445  ...  0.202671  0.016866  0.011997  0.000000   
2  0.004941  0.000000  0.000000  ...  0.005048  0.004892  0.024992  0.000000   
3  0.062969  0.011361  0.031163  ...  0.085938  0.128273  0.307973  0.052985   
4  0.429075  0.000000  0.030611  ...  0.068048  0.418747  0.110148  0.258773   

        605       606

**Code 6: Hybrid**

In [ ]:
import numpy as np
import pandas as pd
import sqlite3
from sklearn.metrics.pairwise import cosine_similarity

def hybrid_recommendation(user_id, movie_id):
    conn = sqlite3.connect('/content/Movies.db')
    cur = conn.cursor()


    cur.execute("SELECT COUNT(*) FROM similarity_ratings WHERE userID = ? AND rating > 0", (user_id,))
    num_ratings = cur.fetchone()[0]

    # Deciding the recommendation method based on the number of ratings
    if num_ratings < 30:
        # Use content-based filtering for users with fewer than 30 ratings
        return get_most_similar_movie(movie_id, cur)
    else:
        # Use collaborative filtering for users with 30 or more ratings
        return collaborative_filtering_recommendations(user_id, cur)

def get_most_similar_movie(movie_id, cur):
    # Fetch the one-hot encoded genre data
    cur.execute(
        'SELECT action, adventure, animation, children, comedy, crime, documentary, drama, fantasy, "film-noir", horror, imax, musical, mystery, romance, "sci-fi", thriller, war, western FROM movies_one_hot_encoding')
    one_zero_vectors = np.array(cur.fetchall(), dtype=int)
    similarity_matrix = np.matmul(one_zero_vectors, np.transpose(one_zero_vectors))
    np.fill_diagonal(similarity_matrix, 0)
    df = pd.DataFrame(similarity_matrix, columns=range(1, len(one_zero_vectors) + 1))
    most_similar_movie_id = df.idxmax(axis=1).loc[movie_id]
    return most_similar_movie_id

def collaborative_filtering_recommendations(user_id, cur):
    # Fetch the ratings array
    cur.execute("SELECT rating FROM similarity_ratings ORDER BY userID, consecutive_movie_id;")
    ratings_vector = np.array(cur.fetchall())
    ratings_matrix = ratings_vector.reshape(610, 9742)  # Ensure these dimensions match your dataset
    ratings_matrix = np.nan_to_num(ratings_matrix)  # Convert NaNs (if any) to zeros

    # Computing the cosine similarity matrix
    cos_sim = cosine_similarity(ratings_matrix)
    np.fill_diagonal(cos_sim, 0)

    df = pd.DataFrame(cos_sim, columns=range(1, len(ratings_matrix) + 1))
    most_similar_users = df.idxmax(axis=1).tolist()

    recommended_movies = []
    if user_id <= len(ratings_matrix):  # Ensure the user_id is valid
        similar_user_id = most_similar_users[user_id - 1]
        cur.execute("SELECT consecutive_movie_id FROM similarity_ratings WHERE userID = ? AND rating >= 4", (similar_user_id,))
        similar_user_movies = [row[0] for row in cur.fetchall()]
        cur.execute("SELECT consecutive_movie_id FROM similarity_ratings WHERE userID = ? AND rating >= 1", (user_id,))
        user_movies = [row[0] for row in cur.fetchall()]
        recos = np.setdiff1d(similar_user_movies, user_movies)
        recommended_movies.extend(recos.tolist())

    return recommended_movies

# Testing the hybrid system with example user and movie IDs
user_id = 53
movie_id = 0  # Ensure this is a valid movie_id
recommendations = hybrid_recommendation(user_id, movie_id)
print(recommendations)


560


**10-20 Users Sample**

In [ ]:
for user_id in sample_user_ids:
    # Collaborative Recommendations
    similar_users = collab_df.loc[user_id].nlargest(3).index.tolist()
    similar_users_ratings = filtered_ratings_pivot.loc[similar_users].mean().nlargest(3).index.tolist()

    # Content-Based Recommendations
    user_genres = ratings_data[(ratings_data['userID'] == user_id) & (ratings_data['consecutive_movie_id'].isin(common_movie_ids))]
    favorite_movies = user_genres[user_genres['rating'] > 4]['consecutive_movie_id']
    if not favorite_movies.empty:
        avg_genre_profile = filtered_genres_pivot.loc[favorite_movies].mean()

        # Computing similarity with all movies based on genre profile
        genre_similarity = cosine_similarity([avg_genre_profile], filtered_genres_pivot)[0]
        top_genre_movies = pd.Series(genre_similarity, index=filtered_genres_pivot.index).nlargest(3).index.tolist()
    else:
        top_genre_movies = filtered_genres_pivot.sum().nlargest(3).index.tolist()

    # Hybrid Recommendations (Combining both approaches)
    hybrid_recos = list(set(similar_users_ratings + top_genre_movies))

    # Storing recommendations with movie names
    recommendations[user_id] = {
        'collaborative': movie_titles.loc[similar_users_ratings]['title'].tolist() if similar_users_ratings else [],
        'content_based': movie_titles.loc[top_genre_movies]['title'].tolist() if top_genre_movies else [],
        'hybrid': movie_titles.loc[hybrid_recos]['title'].tolist() if hybrid_recos else []
    }

# Displaying recommendations
for user, recos in recommendations.items():
    print(f"User {user}:")
    print("  Collaborative:", recos['collaborative'])
    print("  Content-Based:", recos['content_based'])
    print("  Hybrid:", recos['hybrid'])
    print("\n")


User 1:
  Collaborative: ["What's Eating Gilbert Grape ", 'Conspiracy Theory ', 'Whole Wide World, The ']
  Content-Based: ['Toy Story ', 'Jumanji ', 'Grumpier Old Men ']
  Hybrid: ['Whole Wide World, The ', 'Jumanji ', 'Toy Story ', 'Grumpier Old Men ', "What's Eating Gilbert Grape ", 'Conspiracy Theory ']


User 2:
  Collaborative: ['Pat and Mike ', 'Invention of Lying, The ', 'Detour ']
  Content-Based: ['Toy Story ', 'Jumanji ', 'Grumpier Old Men ']
  Hybrid: ['Pat and Mike ', 'Jumanji ', 'Toy Story ', 'Grumpier Old Men ', 'Detour ', 'Invention of Lying, The ']


User 3:
  Collaborative: ['Conspiracy Theory ', 'Kiss Me, Guido ', 'Wonderland ']
  Content-Based: ['Toy Story ', 'Jumanji ', 'Grumpier Old Men ']
  Hybrid: ['Toy Story ', 'Jumanji ', 'Grumpier Old Men ', 'Wonderland ', 'Conspiracy Theory ', 'Kiss Me, Guido ']


User 4:
  Collaborative: ["What's Eating Gilbert Grape ", 'Striptease ', 'Fog, The ']
  Content-Based: ['Toy Story ', 'Jumanji ', 'Grumpier Old Men ']
  Hybrid: ['